In [1]:
# todo:
# merge 6 dfs into a single one and save it into pkl file, add platform as a new column to the df
# go over 3 * 2 files to further clean necessary fields
# go over 3 * 2 files to have a look at basic statistics
# 4.

In [131]:
import os
import json
import pandas as pd
import numpy as np
import re
import pickle

#### liepin JD female first
49 records from liepin for JD records containing "female first" keywords

TODO:  
1. company industry, reduce it to certain amount of words => split into list => done
2. standadized salary units (now is 6-8k·12薪) => into monthly salary => done
3. add "liepin" as the value for a new column "platform" => done
4. company size contains "人", example : 500-999人 => standardize the bin =>done

In [76]:
with open(os.path.dirname(os.path.abspath("."))+'/jd_spider/jd_spider/spiders/output/liepin_jd_female.json', 'r') as f:
    liepin_jd_female = json.load(f)
df_liepin_jd_female = pd.DataFrame(liepin_jd_female)


In [5]:
def liepin_salary(s: str)->(float, float):
    """
    s: string, salary from liepin, example: '30-35k·12薪', last 5 chars k·12薪 are not needed
    return: (low_range, high_range) in thousand RMB
    """ 
    
    if s == '面议':
        return None
    else:
        low, high = s[:-5].split('-') 
        return (float(low)*1000, float(high)*1000)

In [6]:
df_liepin_jd_female[['salary_low', 'salary_high']] = (df_liepin_jd_female['salary_range']
                                                        .str.slice(stop=-5)
                                                        .str.split("-", expand=True)
                                                        .replace('', np.nan)
                                                        .astype(float)
                                                        .multiply(1000)
                                                     )

In [9]:
def liepin_company_size_clean(s):
    """
    s: string, company size field value, clean to get result like "5000-10000"
    return: tuple of integer (low, high), default is (0, 0) for missing value
    """
    m = re.match(r"(?P<size_low>\d+)-(?P<size_high>\d+)", s)
    if m and 'size_low' in m.groupdict() and 'size_high' in m.groupdict():
        return (int(m.groupdict()['size_low']), int(m.groupdict()['size_high']))
    # a tuple of None to enforce the to_list() later separate value into two columns
    return (0, 0) 
    

In [28]:
# to_list() is not stable, cant guarantee to have split the value into two or more lists. if the first value is None, then it will only have one column
df_liepin_jd_female[['company_size_low', 'company_size_high']] = pd.DataFrame(df_liepin_jd_female["company_size"].apply(liepin_company_size_clean).to_list(), index=df_liepin_jd_female.index)




In [31]:
df_liepin_jd_female['platform'] = 'liepin'

In [35]:
df_liepin_jd_female['company_industry'].unique()

array(['专业服务(咨询/财会/法律/翻...', None, '游戏产业', '计算机软件', '百货/批发/零售',
       '基金/证券/期货/投资', '租赁服务', '电子商务', '原材料及加工'], dtype=object)

In [11]:
def extract_first_word(s: str)->str:
    """s: string representing words like company industry, keywords, often separated by different punctuations
        only return the first one
    """
    if s:
        return re.split(r'\W+', s)[0]

In [41]:
df_liepin_jd_female['company_industry'] = df_liepin_jd_female['company_industry'].apply(extract_first_word)

In [44]:
# with open("./liepin_female.pickle", "wb") as f:
#     pickle.dump(df_liepin_jd_female, f)

In [2]:
with open("./liepin_female.pickle", "rb") as f:
    df = pickle.load(f)

In [3]:
df.head()

,url,company_name,company_size,company_industry,company_address,city,district,job_title,job_description,descrimination_content,salary_range,gender_preference,salary_low,salary_high,company_size_low,company_size_high,platform
0,https://www.liepin.com/job/1928849339.shtml,康佩(杭州)企业管理有限公司,江阴市顾山镇人民路3号,专业服务,None,无锡,无锡,人力资源总监,"[职责描述：, 1、根据集团人才发展战略与品牌公司业务战略，制定品牌公司人力资源发展计划和策...",1、本科及以上学历，心理学、人力资源、企业管理等管理类相关专业女性优先； 4、具备优秀的沟通...,30-35k·12薪,female,30000.0,35000.0,0,0,liepin
1,https://www.liepin.com/job/1933052343.shtml,新疆大西洋建设投资有限公司,5000-10000人,None,祈福南湾半岛,佛山,高明区,行政专员,"[职责描述：, 1、协助总部行政中心日常行政工作的开展，包含相关文案、报表的制作等；, 2、...",1、大专及以上，专业不限；播音主持、中文系专业优先考虑 2、女性身高160CM以上，形象气质...,6-8k·12薪,female,6000.0,8000.0,5000,10000,liepin
2,https://www.liepin.com/job/1933052341.shtml,新疆大西洋建设投资有限公司,5000-10000人,None,祈福南湾半岛,佛山,三水区,行政专员,"[职责描述：, 1、协助总部行政中心日常行政工作的开展，包含相关文案、报表的制作等；, 2、...",1、大专毕业，专业不限；播音主持、中文系专业优先考虑 2、女性身高160CM以上，形象气质佳...,6-8k·12薪,female,6000.0,8000.0,5000,10000,liepin
3,https://www.liepin.com/job/1933052339.shtml,新疆大西洋建设投资有限公司,5000-10000人,None,祈福南湾半岛,佛山,顺德区,行政专员,"[职责描述：, 1、协助总部行政中心日常行政工作的开展，包含相关文案、报表的制作等；, 2、...",1、大专毕业，专业不限；播音主持、中文系专业优先考虑 2、女性身高160CM以上，形象气质佳...,6-8k·12薪,female,6000.0,8000.0,5000,10000,liepin
4,https://www.liepin.com/job/1933052337.shtml,新疆大西洋建设投资有限公司,5000-10000人,None,祈福南湾半岛,佛山,南海区,行政专员,"[职责描述：, 1、协助总部行政中心日常行政工作的开展，包含相关文案、报表的制作等；, 2、...",1、大专毕业，专业不限；播音主持、中文系专业优先考虑 2、女性身高160CM以上，形象气质佳...,6-8k·12薪,female,6000.0,8000.0,5000,10000,liepin


In [34]:
df['company_size_high'].dtype

dtype('int64')

In [14]:
def liepin_jd_process(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        df_liepin = pd.DataFrame(data)
        # process salary to monthly salary in K RMB
        # salary from liepin, example: '30-35k·12薪', last 5 chars k·12薪 are not needed, and specail case: 面议
        # transform it into two columns: salary_low, salary_high
        df_liepin[['salary_low', 'salary_high']] = (df_liepin['salary_range']
                                                    .str.slice(stop=-5)
                                                    .str.split("-", expand=True)
                                                    .replace('', np.nan)
                                                    .astype(float)
                                                    .multiply(1000)
                                                    )
        df_liepin[['company_size_low', 'company_size_high']] = pd.DataFrame(df_liepin["company_size"].apply(liepin_company_size_clean).to_list(), index=df_liepin.index)
        df_liepin['company_industry'] = df_liepin['company_industry'].apply(extract_first_word)
        df_liepin['platform'] = 'liepin'
        
        return df_liepin

In [15]:
path = os.path.dirname(os.path.abspath("."))+'/jd_spider/jd_spider/spiders/output/liepin_jd_male.json'
df_liepin_male = liepin_jd_process(path)

In [77]:
df_liepin_jd_female['salary_range'].unique()

array(['30-35k·12薪', '6-8k·12薪', '9-12k·12薪', '20-40k·14薪', '20-35k·14薪',
       '面议', '25-41k·12薪', '9-15k·12薪', '5-8k·12薪', '8-13k·12薪',
       '3-4k·12薪', '4-5k·12薪', '5-10k·12薪', '8-12k·12薪', '10-25k·12薪',
       '12-15k·12薪', '3-8k·12薪', '3-5k·12薪', '5-9k·12薪', '8-10k·12薪',
       '5-15k·12薪', '6-9k·12薪', '10-11k·12薪', '4-8k·12薪', '4-6k·12薪'],
      dtype=object)

In [20]:
######### now clean wu yi jd #####


#### wuyi JD female first
82 records from wuyi for JD records containing "female first" keywords

TODO:  
1. company size, example:  50-150人, 少于50人, 10000人以上, should transform into `company_size_low`, `company_size_high` two colums => done
2. company industry has extra [], multiple words => clean [], only need first word => done
3. job_type, has extra[], multiple words => clean [], only need first word => done
4. keywords: empty[] => None, extra [] needs removal, only need first word => done
5. salary_range, example 6-8千/月, 1-1.5千/月 => should transform into monthly `salary_low`,`salary_high` colums => done
6. jd_date column transform into datetime type => done

In [133]:
with open(os.path.dirname(os.path.abspath("."))+'/jd_spider/jd_spider/spiders/output/wuyi_jd_female.json', 'r') as f:
    wuyi_jd_female = json.load(f)
wuyi_jd_female_df = pd.DataFrame(wuyi_jd_female)

In [134]:
wuyi_jd_female_df.head()

,url,company_name,company_type,company_size,company_industry,company_address,city,district,job_title,job_type,job_description,descrimination_content,keywords,salary_range,gender_preference,age_requirement,jd_date
0,https://jobs.51job.com/guangzhou-byq/126706895...,航友家园（广州）生物科技有限公司,创业公司,50-150人,"[中介服务, 外包服务]",江高神山,广州,白云区,神山汽摩配件组装包吃住200/天,"[普工/操作工, 技工]","[【福利待遇】：长白班，20元/时，包吃包住，最少做2个月。人走结清工资！, 【入职要求】女...",【入职要求】女生16到40岁。18-45周岁（男生），男女都可以（女生优先），身体健康，无精...,[],6-8千/月,female,,11-07
1,https://jobs.51job.com/fuzhou/125588735.html?s...,国网信通亿力科技有限责任公司,国企,None,[通信/电信/网络设备],None,福州,福州,销售工程师,[其他],"[基本任职资格：, 学历要求：大学本科及以上学历。, 专业要求：市场营销、商务管理等相关专业...",4.女生优先，形象气质佳，具有良好的接待、沟通能力强。,[],None,female,,11-07
2,https://jobs.51job.com/foshan-nhq/123009632.ht...,佛山市信优达贸易有限公司,民营公司,少于50人,"[贸易/进出口, 互联网/电子商务]",罗村佛罗路创新产业园A座,佛山,南海区,亚马逊运营助理,"[贸易/外贸专员/助理, 电商专员]","[1, 、负责, Amazon, 账户的日常运营工作，产品上架，推广，优化，物流跟进，售后跟...",、女生优先！男生已招满！,"[亚马逊, 跨境电商, 亚马逊销售, 亚马逊专员, 运营, 电商, 外贸业务员, 外贸]",3.5-5千/月,female,,None
3,https://jobs.51job.com/haikou/122933259.html?s...,用友网络科技股份有限公司海南分公司,民营公司,10000人以上,[计算机软件],海南省海口市国贸大道2号海南时代广场20层A室,海口,海口,实习生-实施实习生,[ERP实施顾问],"[岗位职责：, 1、协助完成客户财务/供应链/HR等软件项目的实施工作。, 2、协助进行客户...",2、女生优先。,[],1-1.5千/月,female,,11-07
4,https://jobs.51job.com/changchun/119802627.htm...,吉林省精科环保科技有限公司,创业公司,少于50人,[环保],吉林省长春经济技术开发区,长春,长春,实验室化验分析员,"[科研人员, 化工实验室研究员/技术员]","[①主要从事水质样品、气体样品、土壤样品、固体废物样品的实验室分析。, ②在实验室分析的同时...",环境科学、环境工程、分析化学及相关专业，本科及以上学历，女生优先；,"[实验室, 技术员, 研究]",3.5-7千/月,female,,11-07


In [32]:
s = '10000人以上'
m = re.match(r"(?P<size_low>\d+)人以上", s)

In [33]:
m.groupdict()

{'size_low': '10000'}

In [135]:
def wuyi_company_size_clean(s):
    """
    s: string, company size field value, clean to get result like "5000-10000"
    return: tuple of integer (low, high), default is (0, 0) for missing value
    """
    if s:
        m = re.match(r"(?P<size_low>\d+)-(?P<size_high>\d+)", s)
        if m and 'size_low' in m.groupdict() and 'size_high' in m.groupdict():
            return (int(m.groupdict()['size_low']), int(m.groupdict()['size_high']))
        # a tuple of None to enforce the to_list() later separate value into two columns
        # series.to_list() is not stable, cant guarantee to have split the value into two or more lists. if the first value is None, then it will only have one column
        m = re.match(r"少于(?P<size_high>\d+)",s)
        if m and 'size_high' in m.groupdict():
            return (0, int(m.groupdict()['size_high']))
        m = re.match(r"(?P<size_low>\d+)人以上", s)
        if m and 'size_low' in m.groupdict():
            return (int(m.groupdict()['size_low']), 0)
        
    return (0, 0) 

In [39]:
s = '少于50人'
wuyi_company_size_clean(s)

(0, 50)

In [136]:
wuyi_jd_female_df[['company_size_low', 'company_size_high']] = pd.DataFrame(wuyi_jd_female_df["company_size"].apply(wuyi_company_size_clean).to_list(), index=wuyi_jd_female_df.index)
        

In [137]:
wuyi_jd_female_df['company_industry'].unique

<bound method Series.unique of 0               [中介服务, 外包服务]
1               [通信/电信/网络设备]
2         [贸易/进出口, 互联网/电子商务]
3                    [计算机软件]
4                       [环保]
               ...          
77    [电子技术/半导体/集成电路, 计算机软件]
78           [电子技术/半导体/集成电路]
79           [电子技术/半导体/集成电路]
80    [计算机软件, 电子技术/半导体/集成电路]
81    [计算机软件, 电子技术/半导体/集成电路]
Name: company_industry, Length: 82, dtype: object>

In [53]:
a='[贸易/进出口, 互联网/电子商务]'
re.search(r'\w+',a).group()

'贸易'

In [65]:
wuyi_jd_female_df['company_industry'].astype('str')

0               ['中介服务', '外包服务']
1                 ['通信/电信/网络设备']
2         ['贸易/进出口', '互联网/电子商务']
3                      ['计算机软件']
4                         ['环保']
                 ...            
77    ['电子技术/半导体/集成电路', '计算机软件']
78             ['电子技术/半导体/集成电路']
79             ['电子技术/半导体/集成电路']
80    ['计算机软件', '电子技术/半导体/集成电路']
81    ['计算机软件', '电子技术/半导体/集成电路']
Name: company_industry, Length: 82, dtype: object

In [138]:
wuyi_jd_female_df['company_industry'] = wuyi_jd_female_df['company_industry'].astype('str').apply(lambda x: re.search(r'\w+',x).group() if x else None)
wuyi_jd_female_df['job_type'] = wuyi_jd_female_df['job_type'].astype('str').apply(lambda x: re.search(r'\w+',x).group() if x else None)
wuyi_jd_female_df['keywords'] = wuyi_jd_female_df['keywords'].astype('str').apply(lambda x: re.search(r'\w+',x).group() if x and x!='[]' else None)

In [139]:
wuyi_jd_female_df.head()

,url,company_name,company_type,company_size,company_industry,company_address,city,district,job_title,job_type,job_description,descrimination_content,keywords,salary_range,gender_preference,age_requirement,jd_date,company_size_low,company_size_high
0,https://jobs.51job.com/guangzhou-byq/126706895...,航友家园（广州）生物科技有限公司,创业公司,50-150人,中介服务,江高神山,广州,白云区,神山汽摩配件组装包吃住200/天,普工,"[【福利待遇】：长白班，20元/时，包吃包住，最少做2个月。人走结清工资！, 【入职要求】女...",【入职要求】女生16到40岁。18-45周岁（男生），男女都可以（女生优先），身体健康，无精...,None,6-8千/月,female,,11-07,50,150
1,https://jobs.51job.com/fuzhou/125588735.html?s...,国网信通亿力科技有限责任公司,国企,None,通信,None,福州,福州,销售工程师,其他,"[基本任职资格：, 学历要求：大学本科及以上学历。, 专业要求：市场营销、商务管理等相关专业...",4.女生优先，形象气质佳，具有良好的接待、沟通能力强。,None,None,female,,11-07,0,0
2,https://jobs.51job.com/foshan-nhq/123009632.ht...,佛山市信优达贸易有限公司,民营公司,少于50人,贸易,罗村佛罗路创新产业园A座,佛山,南海区,亚马逊运营助理,贸易,"[1, 、负责, Amazon, 账户的日常运营工作，产品上架，推广，优化，物流跟进，售后跟...",、女生优先！男生已招满！,亚马逊,3.5-5千/月,female,,None,0,50
3,https://jobs.51job.com/haikou/122933259.html?s...,用友网络科技股份有限公司海南分公司,民营公司,10000人以上,计算机软件,海南省海口市国贸大道2号海南时代广场20层A室,海口,海口,实习生-实施实习生,ERP实施顾问,"[岗位职责：, 1、协助完成客户财务/供应链/HR等软件项目的实施工作。, 2、协助进行客户...",2、女生优先。,None,1-1.5千/月,female,,11-07,10000,0
4,https://jobs.51job.com/changchun/119802627.htm...,吉林省精科环保科技有限公司,创业公司,少于50人,环保,吉林省长春经济技术开发区,长春,长春,实验室化验分析员,科研人员,"[①主要从事水质样品、气体样品、土壤样品、固体废物样品的实验室分析。, ②在实验室分析的同时...",环境科学、环境工程、分析化学及相关专业，本科及以上学历，女生优先；,实验室,3.5-7千/月,female,,11-07,0,50


In [71]:
wuyi_jd_female_df['salary_range'].unique()

array(['6-8千/月', None, '3.5-5千/月', '1-1.5千/月', '3.5-7千/月', '3.5-4.5千/月',
       '4-5千/月', '100元/天', '3-4.5千/月', '0.6-1万/月', '0.8-1万/月', '4.5-6千/月',
       '150元/天', '4.5-5千/月', '0.5-1万/月', '3-5千/月', '1.5-2万/月', '4-6千/月',
       '4.5-8千/月', '0.4-1.2万/月', '3-6千/月'], dtype=object)

In [140]:
def wuyi_salary_clean(s):
    if not s:
        return (np.nan, np.nan)
    elif re.search(r'千/月',s):
        low, high = re.match(r'(\d+\.?\d*)-(\d+\.?\d*)',s).groups()
        return float(low)*1000, float(high)*1000
    elif re.search(r'万/月',s):
        low, high = re.match(r'(\d+\.?\d*)-(\d+\.?\d*)',s).groups()
        return float(low)*10000, float(high)*10000
    elif re.search(r'元/天',s):
        salary = re.match(r'(\d+\.?\d*)', s).group()
        return float(salary)*30, float(salary)*30
    else:
        return (np.nan, np.nan)

In [141]:
wuyi_jd_female_df[['salary_low','salary_high']] = pd.DataFrame(wuyi_jd_female_df['salary_range'].apply(wuyi_salary_clean).tolist(), index=wuyi_jd_female_df.index)

In [142]:
wuyi_jd_female_df['jd_date']

0     11-07
1     11-07
2      None
3     11-07
4     11-07
      ...  
77    10-11
78    09-28
79     None
80    09-27
81    09-27
Name: jd_date, Length: 82, dtype: object

In [143]:
wuyi_jd_female_df['jd_publish_month'] = pd.to_datetime(wuyi_jd_female_df['jd_date'], format='%m-%d', errors='coerce').dt.month.fillna(0).astype(int)

In [150]:
wuyi_jd_female_df[['gender_preference', 'descrimination_content']].head()

,gender_preference,descrimination_content
0,female,【入职要求】女生16到40岁。18-45周岁（男生），男女都可以（女生优先），身体健康，无精...
1,female,4.女生优先，形象气质佳，具有良好的接待、沟通能力强。
2,female,、女生优先！男生已招满！
3,female,2、女生优先。
4,female,环境科学、环境工程、分析化学及相关专业，本科及以上学历，女生优先；


In [146]:
def wuyi_jd_process(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        df[['company_size_low', 'company_size_high']] = pd.DataFrame(df["company_size"].apply(wuyi_company_size_clean).to_list(), index=df.index)
        df['company_industry'] = df['company_industry'].astype('str').apply(lambda x: re.search(r'\w+',x).group() if x else None)
        df['job_type'] = df['job_type'].astype('str').apply(lambda x: re.search(r'\w+',x).group() if x else None)
        df['keywords'] = df['keywords'].astype('str').apply(lambda x: re.search(r'\w+',x).group() if x and x!='[]' else None)
        df[['salary_low','salary_high']] = pd.DataFrame(df['salary_range'].apply(wuyi_salary_clean).tolist(), index=df.index)
        # missing date set as 0
        df['jd_publish_month'] = pd.to_datetime(df['jd_date'], format='%m-%d', errors='coerce').dt.month.fillna(0).astype(int)
        return df

In [151]:
file_path = os.path.dirname(os.path.abspath("."))+'/jd_spider/jd_spider/spiders/output/wuyi_jd_male.json'
wuyi_male_jd_df = wuyi_jd_process(file_path)


In [153]:
wuyi_male_jd_df[['gender_preference', 'descrimination_content']].head()

,gender_preference,descrimination_content
0,male,3、形象气质佳。男生优先，女生有小孩者优先。
1,male,男生优先，女生也可！
2,male,5、较强的适应能力，可驻点省内经销商，进行财务工作指导（广东省内出差，男生优先；女生亦可考虑）
3,male,4、一年以上工作经验，男生优先
4,male,1.要求英语4级—6级，男生优先，条件优秀的女生也可考虑；


In [154]:
wuyi_male_jd_df
with open("./wuyi_male.pickle", "wb") as f:
    pickle.dump(wuyi_male_jd_df, f)

In [155]:
########### zhuobo JD processing ###########

#### zhuobo JD female first
48 records of female first

TODO:
1. company industry example "工艺品、珠宝、钟表", separated with "、" only need the first words => done
2. company size, example "少于50人, 50-200人, None", need to transform into `company_size_low`, `company_size_high` two colums => done
3. job title, some has () and /, remove the words in the () => done
4. job type has [], need to remove the [] and only extract the first words => done
5. keywords: has [], need to remove the [] and only extract the first words, can be none => done
6. salary_rage: example "¥3-5K", need to transform into montly value in two columns  `salary_low`,`salary_high colums` => done
7. age requirements: example, "18-30", "不限", "30岁以下", transform into "age_low", "age_high" two columns => may not needed


In [156]:
with open(os.path.dirname(os.path.abspath("."))+'/jd_spider/jd_spider/spiders/output/zhuobo_jd_female.json', 'r') as f:
    zhuobo_jd_female = json.load(f)
zhuobo_jd_female_df = pd.DataFrame(zhuobo_jd_female)

In [163]:
zhuobo_jd_female_df['company_industry'].unique()

array(['工艺品、珠宝、钟表', '印刷、包装', '机械制造、机电、重工', '建筑施工与工程', '其他生产、制造、加工',
       '纺织品业(服饰、鞋类、家纺用品…)', '咨询、调查、企业管理服务', '橡胶、塑料制品', '贸易、进出口',
       '汽车、摩托车及零配件', '能源', '原材料及加工', '电子技术、半导体、集成电路', '通讯、电信、网络设备',
       '计算机软件', '房地产开发', '金属制品'], dtype=object)

In [164]:
zhuobo_jd_female_df['company_industry'] = zhuobo_jd_female_df['company_industry'].str.split('、').str[0]



In [168]:
zhuobo_jd_female_df[['company_size_low', 'company_size_high']] = pd.DataFrame(zhuobo_jd_female_df["company_size"].apply(wuyi_company_size_clean).to_list(), index=zhuobo_jd_female_df.index)
        

In [221]:
def job_title_cleans(s):
    if s:
        s = s.split('/')[0] # remove '/', '商务跟单/助理' => '商务跟单'
        p = r'(\w*)(?<!（|）)(\w*)' # '采购（急聘！）' => '采购'
        res = re.match(p, s)
        if res:
            return res.group()
        p = r'\(|\)|（|）'  # '(女生优先)销售代表' => '销售代表'
        return re.split(p, s)[-1]
    return None

In [223]:
zhuobo_jd_female_df['job_title'] = zhuobo_jd_female_df['job_title'].apply(job_title_cleans)

In [227]:
zhuobo_jd_female_df['job_type'] = zhuobo_jd_female_df['job_type'].astype('str').apply(lambda x: re.search(r'\w+',x).group() if x and x!='[]' else None)

In [230]:
zhuobo_jd_female_df['keywords'] = zhuobo_jd_female_df['keywords'].astype('str').apply(lambda x: re.search(r'\w+',x).group() if x and x!='[]' else None)

In [247]:
def zhuobo_salary_clean(s):
    if s:
        res = re.match(r'¥(\d+\.?\d*)-(\d+\.?\d*)K',s)
        low, high = res.groups() if res else (np.nan, np.nan)
        return float(low)*1000, float(high)*1000
    return (np.nan, np.nan)

In [249]:
zhuobo_jd_female_df[['salary_low','salary_high']] = pd.DataFrame(zhuobo_jd_female_df['salary_range'].apply(zhuobo_salary_clean).tolist(), index=zhuobo_jd_female_df.index)
        

In [254]:
zhuobo_jd_female_df['descrimination_content'].head()

0    7.女性优先 4、有客服经验以及手机通讯相关工作经验者优先， 5、个性开朗，善于沟通表达，具...
1    1、大专以上学历，女性优先，至少一年以上品质工程师工作经验。 5、全面管理公司与品质相关工作...
2                                        3.24周岁内，女性优先。
3    1、年龄在30周岁以下，女性优先； 3、2年以上人事行政方面的工作经验，有质量管理体系经验的...
4    1、熟悉Excel办公软件电脑操作，会AI与PS软件优先考虑。 欢迎应届毕业生和有经验人员加...
Name: descrimination_content, dtype: object

In [256]:
def zhuobo_jd_process(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        df['company_industry'] = df['company_industry'].str.split('、').str[0]
        df['job_title'] = df['job_title'].apply(job_title_cleans)
        df['job_type'] = df['job_type'].astype('str').apply(lambda x: re.search(r'\w+',x).group() if x and x!='[]' else None)
        df['keywords'] = df['keywords'].astype('str').apply(lambda x: re.search(r'\w+',x).group() if x and x!='[]' else None)
        df[['salary_low','salary_high']] = pd.DataFrame(df['salary_range'].apply(zhuobo_salary_clean).tolist(), index=df.index)
        
        
        return df

In [258]:
file_path = os.path.dirname(os.path.abspath("."))+'/jd_spider/jd_spider/spiders/output/zhuobo_jd_male.json'

In [260]:
zhuobo_male_df = zhuobo_jd_process(file_path)

In [263]:
zhuobo_male_df['salary_range'].head()

0      ¥3-5K
1     ¥8-16K
2    ¥4.5-6K
3    ¥3.5-6K
4     ¥6-12K
Name: salary_range, dtype: object